In [2]:
import pandas as pd
import numpy as np
import datetime
now = datetime.datetime.now()

euro = 4.227
distribution = 76

Auchan = pd.read_excel('Pobrane/Kwiecien_2018.xlsx', skiprows=2, na_filter=True)

Auchan.drop(["Distribution Weighted","Unnamed: 15"], axis=1, inplace=True)
Auchan.dropna(axis=0,how="all",inplace=True)

Auchan["MS_category"] = Auchan["Turnover"]/Auchan["Turnover Auchan"]
Auchan["Distribution"] = (Auchan["Distribution Numerical"]/Auchan["Distribution Numerical"].max())*distribution
Auchan["Sell_Out_Value"] = Auchan["Turnover"]*euro
Auchan["Sell_Out_Value_LY"] = (Auchan["Turnover"] - Auchan["Variance Turnover"])*euro
Auchan["Sell_Out_Evolution"] = Auchan["Sell_Out_Value"]/Auchan["Sell_Out_Value_LY"] - 1

EAN = []
Product = Auchan["Product"].tolist()
for el in Product:
    if str(el) == '*** ALL PRODUCT':
        ean = 'nan'
    else:
        index = str(el).index("-")
        ean = str(el)[:index-1]
        
    EAN.append(ean)

Auchan["EAN"] = EAN    

Auchan_Loreal = Auchan[Auchan["EAN"]!="nan"][["EAN","Sub-Department","Category","Range","Product","Sell_Out_Value","Sell_Out_Value_LY",
                              "Sell_Out_Evolution","MS_category","Distribution"]]

categories = Auchan[["Sub-Department","Category", "Turnover Auchan"]]
categories = categories[categories["Sub-Department"]!="*** ALL SUB DEPARTMENT"][categories["Category"]=="*** ALL CATEGORY"]
categories.drop("Category", axis=1,inplace=True)
categories["Turnover Auchan"] = categories["Turnover Auchan"]*euro

Auchan_Loreal = Auchan_Loreal.merge(categories,on="Sub-Department", how='left')
Auchan_Loreal["MS_Department"] = Auchan_Loreal["Sell_Out_Value"]/Auchan_Loreal["Turnover Auchan"]

Auchan_Loreal.columns = ['EAN', 'Category', 'Segment', 'Range', 'Product',
       'Sell_Out_Value', 'Sell_Out_Value_LY', 'Sell_Out_Evolution',
       'MS_segment', 'Distribution', 'Category_Turnover', 'MS_Category']

writer = pd.ExcelWriter('Auchan_Sell_Out_'+str(now.year)+'_'+str(now.month)+'.xlsx')
Auchan_Loreal.to_excel(writer,'Sell_Out')

writer.save()


/home/kodolamacz/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:37: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
